In [1]:
import pydrake

# 线性规划

$$
\begin{aligned} 
    \min_x \;c^Tx + d\\ 
    \text{subject to } Ax\leq b 
\end{aligned}
$$

drake 支持一些线性规划的求解器，包括 SCS、Gurobi、MOSEK™ 等，请参阅 [Doxygen 文档]( https://drake.mit.edu/doxygen_cxx/group__solvers.html) 查看完整列表。注意，一些商业解决方案（如 Gurobi 和 MOSEK™) 未包含在预编译的 drake 二进制文件中. 关于求解优化问题相关的所有函数，请参考 [Doxygen 文档](https://drake.mit.edu/doxygen_cxx/classdrake_1_1solvers_1_1_mathematical_program.html)。

## 线性代价函数

In [2]:
from pydrake.solvers import MathematicalProgram, Solve
import numpy as np

prog = MathematicalProgram()
x = prog.NewContinuousVariables(2, 'x')

# Linear cost 1
cost1 = prog.AddLinearCost(x[0] + 3 * x[1] + 2)
print(f"cost1: {cost1}")
print(prog.linear_costs()[0])

# Linear cost 2
cost2 = prog.AddLinearCost(2 * x[1] + 3)
print(f"number of linear cost objects: {len(prog.linear_costs())}")

# Linear cost 3 by coefficient
cost3 = prog.AddLinearCost([3, 4], 5, x)  # 3 * x[0] + 4 * x[1] + 5
print(f"cost3: {cost3}")

# Linear cost 4 by AddCost. drake will analyze the structure of the expression, determine whether the expression is linear
print(f"number of linear cost objects before calling AddCost: {len(prog.linear_costs())}")
cost4 = prog.AddCost(x[0] + 3 * x[1] + 5)
print(f"number of linear cost objects after calling AddCost: {len(prog.linear_costs())}")

cost1: LinearCost (2 + x(0) + 3 * x(1))
LinearCost (2 + x(0) + 3 * x(1))
number of linear cost objects: 2
cost3: LinearCost (5 + 3 * x(0) + 4 * x(1))
number of linear cost objects before calling AddCost: 3
number of linear cost objects after calling AddCost: 4


## 线性约束条件

有三种：

* $ lower \le x \le upper $.
* $Ax = b$.
* $lower \le Ax \le upper$.

In [7]:
prog = MathematicalProgram()
x = prog.NewContinuousVariables(2, "x")
y = prog.NewContinuousVariables(3, "y")

bounding_box1 = prog.AddConstraint(x[0] >= 1)
print(
    f"number of bounding box constraint objects: {len(prog.bounding_box_constraints())}"
)

bounding_box2 = prog.AddLinearConstraint(x[1] <= 2)
print(
    f"number of bounding box constraint objects: {len(prog.bounding_box_constraints())}"
)

linear_eq1 = prog.AddConstraint(x[0] + y[1] == 3.0)
print(
    f"number of linear equality constraint objects: {len(prog.linear_equality_constraints())}"
)

linear_eq2 = prog.AddLinearConstraint(x[1] + 2 * y[2] == 1)
print(
    f"number of linear equality constraint objects: {len(prog.linear_equality_constraints())}"
)

linear_ineq1 = prog.AddConstraint(x[0] + 3 * x[1] + 2 * y[2] <= 4)
print(
    f"number of linear inequality constraint objects: {len(prog.linear_constraints())}"
)

linear_ineq2 = prog.AddLinearConstraint(x[1] + 4 * y[1] >= 2)
print(
    f"number of linear inequality constraint objects: {len(prog.linear_constraints())}"
)

try:
    prog.AddLinearConstraint(x[0] ** 2 == 2)
except RuntimeError as err:
    print(err.args)

# more efficient for "lower ≤ Ax ≤ upper", e.g.
# Add a linear constraint 2x[0] + 3x[1] <= 2, 1 <= 4x[1] + 5y[2] <= 3.
# This is equivalent to lower <= A * [x;y[2]] <= upper with
# lower = [-inf, 1], upper = [2, 3], A = [[2, 3, 0], [0, 4, 5]].
linear_constraint = prog.AddLinearConstraint(
    A=[[2.0, 3.0, 0], [0.0, 4.0, 5.0]],
    lb=[-np.inf, 1],
    ub=[2.0, 3.0],
    vars=np.hstack((x, y[2])),
)
print(linear_constraint)

# Add a bounding box constraint -1 <= x[0] <= 2, 3 <= x[1] <= 5
bounding_box3 = prog.AddBoundingBoxConstraint([-1, 3], [2, 5], x)
print(bounding_box3)

# Add a bounding box constraint 3 <= y[i] <= 5 for all i.
bounding_box4 = prog.AddBoundingBoxConstraint(3, 5, y)
print(bounding_box4)

# Add a linear equality constraint 4 * x[0] + 5 * x[1] == 1，比 AddConstraint 和 AddLinearConstraint 更准确，更快
linear_eq3 = prog.AddLinearEqualityConstraint(np.array([[4, 5]]), np.array([1]), x)
print(linear_eq3)

number of bounding box constraint objects: 1
number of bounding box constraint objects: 2
number of linear equality constraint objects: 1
number of linear equality constraint objects: 2
number of linear inequality constraint objects: 1
number of linear inequality constraint objects: 2
('Formula (pow(x(0), 2) == 2) is non-linear.',)
LinearConstraint
-inf <= (2 * x(0) + 3 * x(1)) <= 2
1 <= (4 * x(1) + 5 * y(2)) <= 3



## 求解算例

$$
\begin{aligned} 
    \min_x \;& -3x[0] - x[1] - 5x[2] -x[3] + 2\\ 
    \text{subject to} \;& 3x[0] + x[1] + 2x[2] = 30 \\
    & 2x[0] + x[1] + 3x[2] + x[3] >= 15 \\
    & 2x[1] + 3x[3] <= 25 \\
    & -100 <= x[0] + 2x[2] <= 40 \\
    & x[0], x[1], x[2], x[3] >= 0, x[1] <= 10 \\
\end{aligned}
$$

In [15]:
prog = MathematicalProgram()

x = prog.NewContinuousVariables(4)

prog.AddLinearCost(-3 * x[0] - x[1] - 5 * x[2] - x[3] + 2)

prog.AddLinearEqualityConstraint([[3, 1, 2, 0]], [30], x)
prog.AddLinearConstraint(
    A=[[2, 1, 3, 1], [0, 2, 0, 3], [1, 0, 2, 0]],
    lb=[15, -np.inf, -100],
    ub=[np.inf, 25, 40],
    vars=x,
)
prog.AddBoundingBoxConstraint([0, 0, 0, 0], [np.Inf, 10, np.Inf, np.Inf], x)

result = Solve(prog)
print(f"Is solved successfully: {result.is_success()}")
print(f"x optimal value: {result.GetSolution(x)}")
print(f"optimal cost: {result.get_optimal_cost()}")


Is solved successfully: True
x optimal value: [ 0.          0.         15.          8.33333333]
optimal cost: -81.33333333333333
